### A Practical Introduction to Integer Programming
#### Igor Ferst, PyOhio 2019

Code based on:
https://github.com/google/or-tools/blob/stable/ortools/linear_solver/samples/simple_mip_program.py

Install `ortools` if you haven't already.

In [ ]:
import sys
!{sys.executable} -m pip install --user ortools

Instantiate a solver.

In [1]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('PyOhio 2019', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Define all variables, one for each vehicle/route pair.

In [2]:
x = {
    'sedan,1': solver.IntVar(0.0, 1.0, 'sedan,1'),
    'sedan,2': solver.IntVar(0.0, 1.0, 'sedan,2'),
    'sedan,3': solver.IntVar(0.0, 1.0, 'sedan,3'),
    'sedan,4': solver.IntVar(0.0, 1.0, 'sedan,4'),
    'van,1': solver.IntVar(0.0, 1.0, 'van,1'),
    'van,2': solver.IntVar(0.0, 1.0, 'van,2'),
    'van,3': solver.IntVar(0.0, 1.0, 'van,3'),
    'van,4': solver.IntVar(0.0, 1.0, 'van,4'),    
    'bus,1': solver.IntVar(0.0, 1.0, 'bus,1'),
    'bus,2': solver.IntVar(0.0, 1.0, 'bus,2'),
    'bus,3': solver.IntVar(0.0, 1.0, 'bus,3'),
    'bus,4': solver.IntVar(0.0, 1.0, 'bus,4'),    
}

print(('Total number of variables: %d' % solver.NumVariables()))

Total number of variables: 12


Define all constraints.

In [3]:
# Coverage constraints: every route should have exactly one vehicle assigned to it.

solver.Add( x['sedan,1'] + x['van,1'] + x['bus,1'] == 1 )
solver.Add( x['sedan,2'] + x['van,2'] + x['bus,2'] == 1 )
solver.Add( x['sedan,3'] + x['van,3'] + x['bus,3'] == 1 )
solver.Add( x['sedan,4'] + x['van,4'] + x['bus,4'] == 1 )

# Capacity constraints: a vehicle assigned to a route must have sufficient capacity.

solver.Add( x['sedan,1'] == 0 )
solver.Add( x['sedan,2'] == 0 )
solver.Add( x['van,1'] == 0 )

# Overlap constraints: two routes that overlap in time can’t use the same vehicle.

solver.Add( x['sedan,1'] + x['sedan,2'] <= 1 ) 
solver.Add( x['van,1'] + x['van,2'] <= 1 ) 
solver.Add( x['bus,1'] + x['bus,2'] <= 1 ) 

solver.Add( x['sedan,3'] + x['sedan,4'] <= 1 ) 
solver.Add( x['van,3'] + x['van,4'] <= 1 ) 
solver.Add( x['bus,3'] + x['bus,4'] <= 1 ) 

print(('Total number of constraints: %d' % solver.NumConstraints()))

Total number of constraints: 13


Define function to be minimized (objective function).

In [4]:
# Minimize the sum, over all routes, of the capacity of the vehicle assigned to that route. 

solver.Minimize(
    4 * x['sedan,1'] + 6 * x['van,1'] + 10 * x['bus,1'] + 
    4 * x['sedan,2'] + 6 * x['van,2'] + 10 * x['bus,2'] + 
    4 * x['sedan,3'] + 6 * x['van,3'] + 10 * x['bus,3'] + 
    4 * x['sedan,4'] + 6 * x['van,4'] + 10 * x['bus,4']    
)

Now we're ready to solve!

In [5]:
result_status = solver.Solve()

# Confirm the problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# Confirm the solution looks legit (when using solvers others than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended).
assert solver.VerifySolution(1e-7, True)

# Get vehicle assignments.
print('Assignments:')
for assignment in [x_var for x_var in x.values() if x_var.solution_value() == 1.0]:
    print(assignment)

# The objective value of the solution (i.e. minimal value of the function to be minimized)
print(('\nMinimal objective value: %f' % solver.Objective().Value()))

Assignments:
sedan,4
van,2
van,3
bus,1

Minimal objective value: 26.000000
